# PROYECTO NO. 1 Data Science

#### 1. Crear un entorno de anaconda con los paquetes que considere necesarios

In [17]:
#Inciaremos importando las librerías a usar en el proyecto
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
from prettytable import PrettyTable

#### Usando slicing con NumPy separar los datos en 2 datasets: entrenamiento(80 %) y validación y pruebas(20 %).

In [31]:
#En base a las indicaciones se procede a leer el archivo necesario y hacer un slicing array con numpy 
dataset = np.load("proyecto_training_data.npy")
num_nans = np.count_nonzero(np.isnan(dataset))
print("Hay", num_nans, "valores NaN en la matriz.")
size_to_train = int(len(dataset) * 0.8)
data_train = dataset[:size_to_train]
data_test = dataset[size_to_train:]
total_data = dataset.shape
print("El tamaño del set de datos total es de: " + str(total_data))
size_of_train_data = data_train.shape
print("El tamaño del set de datos para el entrenamiento es de: " + str(size_of_train_data))
size_of_data_test= data_test.shape
print("El tamaño del set de datos para las pruebas es de: " + str(size_of_data_test))

Hay 259 valores NaN en la matriz.
El tamaño del set de datos total es de: (1460, 6)
El tamaño del set de datos para el entrenamiento es de: (1168, 6)
El tamaño del set de datos para las pruebas es de: (292, 6)


In [24]:
#Debido a que el proyecto solicita hacerlo con Numpy slicing, sin embargo yo recomendaría numpy split, dejo el código
data_train2, data_test2 = np.split(dataset, [size_to_train])

#### Análisis exploratorio de datos: Para cada variable en el dataset calcular:
 - media
 - valor máximo
 - valor míınimo
 - rango(peak to peak, no el rango del tensor que por ser vector sabemos que es 1)
 - desviacióon estándar.

In [39]:
#Se le agregan los nombres a cada una de las columnas
dataset_pandas = pd.DataFrame(dataset, columns = ["SalePrice","OverallQual","1stFlrSF","TotRmsAbvGrd","YearBuilt","LotFrontage"])
pd.options.display.float_format = "{:,.2f}".format
num = [ float('%.2f' % elem) for elem in list(np.arange(0, 1, 0.1)) ]
#Por facilidad el análisis se realizara mediante pandas con la función describe en donde se mostraran los percentiles con aumento de 0.1
un_stats = dataset_pandas.describe(percentiles=num)
print('\nInformación estadística:')
un_stats


Información estadística:


,SalePrice,OverallQual,1stFlrSF,TotRmsAbvGrd,YearBuilt,LotFrontage
count,"1,460.00","1,460.00","1,460.00","1,460.00","1,460.00","1,201.00"
mean,"180,921.20",6.10,"1,162.63",6.52,"1,971.27",70.05
std,"79,442.50",1.38,386.59,1.63,30.20,24.28
min,"34,900.00",1.00,334.00,2.00,"1,872.00",21.00
0%,"34,900.00",1.00,334.00,2.00,"1,872.00",21.00
10%,"106,475.00",5.00,756.90,5.00,"1,924.90",44.00
20%,"124,000.00",5.00,848.00,5.00,"1,947.80",53.00
30%,"135,500.00",5.00,915.70,6.00,"1,958.00",60.00
40%,"147,000.00",6.00,"1,000.20",6.00,"1,965.00",63.00
50%,"163,000.00",6.00,"1,087.00",6.00,"1,973.00",69.00


In [45]:
#Se asume como rango peak a la diferencia entre el valor máximo y mínimo. Se puede utilizar la función ptp, en donde se deben convertir los nan a 0 primero, pero esto puede conllevar un error
rango_peak = dataset_pandas.apply(lambda x: np.nan_to_num(x.to_numpy()).ptp())
print("El rango para cada una de las columnas es de:\n", rango_peak)

El rango para cada una de las columnas es de:
 SalePrice      720,100.00
OverallQual          9.00
1stFlrSF         4,358.00
TotRmsAbvGrd        12.00
YearBuilt          138.00
LotFrontage        313.00
dtype: float64


In [47]:
#En base a lo anterior para este caso se recomienda hacer lo siguiente para calcular los rangos
rango_peak = un_stats.loc["max"]-un_stats.loc["min"]
print("El rango para cada una de las columnas es de:\n", rango_peak)

El rango para cada una de las columnas es de:
 SalePrice      720,100.00
OverallQual          9.00
1stFlrSF         4,358.00
TotRmsAbvGrd        12.00
YearBuilt          138.00
LotFrontage        292.00
dtype: float64


#### 4. Para cada variable en el dataset usar seaborn para graficar un histograma de la variable.

In [41]:
dataset_pandas

,SalePrice,OverallQual,1stFlrSF,TotRmsAbvGrd,YearBuilt,LotFrontage
0,"208,500.00",7.00,856.00,8.00,"2,003.00",65.00
1,"181,500.00",6.00,"1,262.00",6.00,"1,976.00",80.00
2,"223,500.00",7.00,920.00,6.00,"2,001.00",68.00
3,"140,000.00",7.00,961.00,7.00,"1,915.00",60.00
4,"250,000.00",8.00,"1,145.00",9.00,"2,000.00",84.00
...,...,...,...,...,...,...
1455,"175,000.00",6.00,953.00,7.00,"1,999.00",62.00
1456,"210,000.00",6.00,"2,073.00",7.00,"1,978.00",85.00
1457,"266,500.00",7.00,"1,188.00",9.00,"1,941.00",66.00
1458,"142,125.00",5.00,"1,078.00",5.00,"1,950.00",68.00
